# **Lab12** Implementation of Linear Regression and Logistic Regression from MLlib using Pyspark.  

Credit note to [Asif Anif](https://github.com/asifahmed90/pyspark-ML-in-Colab) for the Jupyter notebook on an example of implementation of MLlib on Pyspark using LInear Regression and [Susan Li](https://github.com/susanli2016/PySpark-and-MLlib/blob/master/Logistic_Regression_Titanic.ipynb) for the Jupyter notebook on an example of implementation of MLlib on Pyspark using Logistic Regression.   

Remember in Lab 10, we did include the following description of SparkSession. 

- **SparkSession** serves as entry point for Spark 2.0 and later. It combines the functionality of SparkContext, SQLContext, HiveContext and StreamingContext. In another words, you can work on DataFrames and Datasets, perform structured query (SQL), Machine Learning algorithms and streaming. 
- In this notebook, we will look into the **working with Machine Learning algorithm** that is supported through Spark Session. 

In [1]:
# Set the PySpark environment variables. 
# The Spark's path depends on where you have saved your Spark. Refer to earlier guide in PDF 
import os
os.environ['SPARK_HOME'] = "/Applications/Spark"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
os.environ['PYSPARK_PYTHON'] = 'python'

In [2]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MyLR") \
    .getOrCreate()

# Get the SparkContext from the SparkSession
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/16 23:27:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Linear Regression Model


Linear Regression model is one the oldest and widely used machine learning approach which assumes a relationship between dependent and independent variables. Linear Regression consists of the best fitting line through the scattered points on the graph and the best fitting line is known as the regression line.

The goal of this exercise to predict the housing prices by the given features. Let's predict the prices of the Boston Housing dataset by considering **MEDV** as the output variable and all the other variables as input.


In [3]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

dataset = spark.read.csv('house.csv',inferSchema=True, header =True)

Notice that we used InferSchema inside read.csv mofule. InferSchema enables us to infer automatically different data types for each column. Let us print look into the dataset to see the data types of each column:

In [4]:
dataset.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



Now that we have uploaded the dataset, we can start analyzing. 
For our linear regression model we need to import two modules from Pyspark i.e. Vector Assembler and Linear Regression. Vector Assembler is a transformer that assembles all the features into one vector from multiple columns that contain type double. We could have used StringIndexer if any of our columns contains string values to convert it into numeric values. Luckily, the BostonHousing dataset only contains double values, so we don't need to worry about StringIndexer for now.

Next step is to combine all the features from different columns into a single column and let's call this new vector column as 'Attributes' in the outputCol.

In [5]:
#Input all the features in one vector column
assembler = VectorAssembler(inputCols=['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'b', 'lstat'], outputCol = 'Attributes')

output = assembler.transform(dataset)

#Input vs Output
finalized_data = output.select("Attributes","medv")

finalized_data.show()

24/09/16 23:28:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----+
|          Attributes|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



Here, 'Attributes' are in the input features from all the columns and 'medv' is the target column.
Next, we should split the training and testing data according to our dataset (0.8 and 0.2 in this case).

In [6]:
#Split training and testing data
train_data,test_data = finalized_data.randomSplit([0.8,0.2])

regressor = LinearRegression(featuresCol = 'Attributes', labelCol = 'medv')

#Learn to fit the model from training set
regressor = regressor.fit(train_data)

#To predict the prices on testing set
pred = regressor.evaluate(test_data)

#Predict the model
pred.predictions.show()

24/09/16 23:28:49 WARN Instrumentation: [073b69e2] regParam is zero, which might cause numerical instability and overfitting.
24/09/16 23:28:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/09/16 23:28:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/09/16 23:28:50 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


+--------------------+----+------------------+
|          Attributes|medv|        prediction|
+--------------------+----+------------------+
|[0.01301,35.0,1.5...|32.7|30.040278191869803|
|[0.01381,80.0,0.4...|50.0| 39.70922247085762|
|[0.01501,90.0,1.2...|50.0| 43.93824479631349|
|[0.01951,17.5,1.3...|33.0|23.136252964534535|
|[0.02009,95.0,2.6...|50.0|  41.8513795239943|
|[0.02055,85.0,0.7...|24.7| 25.10703644591642|
|[0.02187,60.0,2.9...|31.1| 31.53206330288802|
|[0.02543,55.0,3.7...|23.9| 27.60125687555751|
|[0.02729,0.0,7.07...|34.7|30.633877965565304|
|[0.02731,0.0,7.07...|21.6|25.288522929021475|
|[0.03306,0.0,5.19...|20.6|21.937314350444524|
|[0.03548,80.0,3.6...|20.9|  21.7563333458766|
|[0.03551,25.0,4.8...|22.9|25.258806205494043|
|[0.03705,20.0,3.3...|35.4| 33.89584565557524|
|[0.03738,0.0,5.19...|20.7|21.546134274528146|
|[0.04203,28.0,15....|22.9|28.885184332151283|
|[0.04379,80.0,3.3...|19.4| 25.42353961881318|
|[0.04684,0.0,3.41...|22.6|27.027740604164258|
|[0.04819,80.

We can also print the coefficient and intercept of the regression model by using the following command:

In [8]:
#coefficient of the regression model
coeff = regressor.coefficients

#X and Y intercept
intr = regressor.intercept

print ("The coefficient of the model is : %a" %coeff)
print ("The Intercept of the model is : %f" %intr)


The coefficient of the model is : DenseVector([-0.0843, 0.0433, 0.0405, 3.2593, -19.1786, 2.9877, 0.0166, -1.3881, 0.2776, -0.0106, -0.9466, 0.006, -0.6619])
The Intercept of the model is : 42.923890


In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="medv", predictionCol="prediction")

# Mean Absolute Error
mae = eval.evaluate(pred.predictions, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred.predictions, {eval.metricName: "r2"})
print("r2: %.3f" %r2)



MAE: 3.826
r2: 0.712


In [10]:
spark.stop()

### Still remember what do the above metrics refer to ? 

# Logistic Regression Model

The goal of this exercise to predict whether the passenger survived or did not survive by the given features. Let's predict the prices of the Boston Housing dataset by considering **Survived** as the output variable and all the other variables as input. Some of the descriptions to the codes used in subsequent code cells can be found in earlier code cells.

In [11]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

spark = SparkSession.builder.appName('MyLogreg').getOrCreate()
df = spark.read.csv('titanic.csv', inferSchema = True, header = True)
df.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



In [12]:
df.printSchema() 

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [13]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [14]:
my_col = df.select(['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked'])

In [15]:
final_data = my_col.na.drop()

In [16]:
from pyspark.ml.feature import (VectorAssembler, StringIndexer, VectorIndexer, OneHotEncoder)
#StringIndexer encodes a string column of labels to a column of label indices
#A one-hot encoder that maps a column of category indices to a column of binary vectors, with at most a single one-value per row that indicates the input category index.
gender_indexer = StringIndexer(inputCol = 'Sex', outputCol = 'SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex', outputCol = 'SexVec')

In [17]:
embark_indexer = StringIndexer(inputCol = 'Embarked', outputCol = 'EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol = 'EmbarkIndex', outputCol = 'EmbarkVec')

In [18]:
assembler = VectorAssembler(inputCols = ['Pclass', 'SexVec', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkVec'], outputCol = 'features')

In [19]:
from pyspark.ml import Pipeline

log_reg = LogisticRegression(featuresCol = 'features', labelCol = 'Survived')

In [20]:
pipeline = Pipeline(stages = [gender_indexer, embark_indexer, 
                             gender_encoder, embark_encoder,
                             assembler, log_reg])

In [21]:
#70% Training and 30% Testing. You can change this ratio
train, test = final_data.randomSplit([0.7, 0.3])


In [22]:
fit_model = pipeline.fit(train)

In [23]:
results = fit_model.transform(test)

In [24]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

eval = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction', labelCol = 'Survived')
AUC = eval.evaluate(results)
AUC

0.8386986301369862

In [25]:
spark.stop()